# For Getting channels log

## Import modules

In [ ]:
import os
from slack_sdk import WebClient
import pandas as pd
import numpy as np
import datetime
from collections import OrderedDict
import openpyxl
import urllib

## WebApp settings

In [ ]:
proxy = urllib.request.getproxies().get('http')
print(proxy)
client = WebClient(token=os.environ['TEST_CH_BOT_TOKEN'], proxy=proxy)
uclient = WebClient(token=os.environ['TEST_CH_USER_TOKEN'], proxy=proxy)

## get log of messages (without replies)

### get users info for retrive DM log

#### create table contains user ids and names

In [ ]:
# get users data
users_info = uclient.users_list().get('members')
df_users_info = pd.json_normalize(users_info)

# create list of user_id and names
user_id_names = []
for i in zip(df_users_info['id'], df_users_info['real_name']):
    user_id_names.append(i)
df_user_id_names = pd.DataFrame(user_id_names,columns=['id', 'real_name']).fillna('deleted user').astype({'real_name': str})
df_user_id_names

#### specify user whom you want to retrive logs of DM with

In [ ]:
# select number on the left of 'id' column above table
usr_idx = 4 # example: 'test_user'

dm_user_id = df_user_id_names['id'][usr_idx]
dm_user_name = df_user_id_names['real_name'][usr_idx]
print('Specified user: '+dm_user_name)

#### retrive id of DM channel

In [ ]:
dm_info = uclient.conversations_open(users=dm_user_id) 
print(dm_info)
dm = dm_info.get('channel')
print(dm['id'])

### get messages log for specified ch

#### retrive raw data and create table of messages log

In [ ]:
# get DM logs    
log_dm = uclient.conversations_history(channel=dm['id']) # it is raw data without replies
log_dm_ms = log_dm['messages']
list_log = []
df_log_dm_ms = pd.json_normalize(log_dm_ms)
i = 0

# can get over 100 messages
if log_dm['has_more'] == True:
    while log_dm['has_more'] == True:
        log_dm = uclient.conversations_history(channel=dm['id'], cursor=log_dm['response_metadata']['next_cursor'])
        log_dm_ms_n = log_dm['messages']
        list_log.append(log_dm['messages'])

        df_log_dm_ms_n = pd.json_normalize(log_dm_ms_n)
        df_log_dm_ms = df_log_dm_ms.append(df_log_dm_ms_n)
        i += 1
print(i)

flat = [x for row in list_log for x in row]
log_dm_ms.extend(flat) # log_dm_ms can be stored over 100 logs
print(len(log_dm_ms)) # num of messages
df_log_dm_ms.columns

#### select info what you want to retrive and format table

In [ ]:
# select columns to use
reindex_col = ['type', 'thread_ts', 'ts', 'user', 'text', 'reply_count', 'reply_users_count']
df_log_dm_ms = df_log_dm_ms.reindex(columns=reindex_col)
df_log_dm_ms

### (in MESSAGES) retrive attached files and create table contains file name and download url

In [ ]:
# extract value of files key
files = []
for i in df_log_dm_ms.index:
    if 'files' in log_dm_ms[i].keys():
        files.append(log_dm_ms[i]['files'][0])
    else:
        files.append({'name': 'None', 'url_private_download': 'None'})  

# create df of files and rename columns
df_dm_files = pd.json_normalize(files)
df_dm_files = df_dm_files[['name', 'url_private_download']]
df_dm_files = df_dm_files.rename(columns={'name': 'FileName', 'url_private_download': 'FileURL'})
df_dm_files

### combine MESSAGE table with file rep table and sort by ts

In [ ]:
df_dm_log_ms = pd.concat([df_log_dm_ms, df_dm_files], axis=1).sort_values(by='ts').reset_index().drop(columns='index')
df_dm_log_ms 

---
## get replies to associate with each replies by parent messages

### search messages including replies and retrive reps with parent messages

In [ ]:
channel_id = dm['id']
threads = []
for i in df_dm_log_ms.index:
    if np.isnan(df_dm_log_ms['reply_count'][i]) == False:
        thr = uclient.conversations_replies(channel=channel_id, ts=df_dm_log_ms['thread_ts'][i]) # it is raw data of replies
        thr_ms = thr.get('messages')
        threads.append(thr_ms)

In [ ]:
reps = []
for i in range(len(threads)):
    rep = threads[i]
    for j in range(len(rep)):
        #print(rep[i])
        reps.append(rep[j])
print(len(reps))

### following steps for create table contains messages and reps
#### create reply table
#### (in REPLIES) retrive attached files and create table contains file name and download url
#### combine REPLY table with FILE table and sort by timestamp

In [ ]:
if len(reps) != 0:
    # create reply table
    reindex_col = ['type', 'thread_ts', 'ts', 'user', 'text', 'reply_count', 'reply_users_count']
    df_dm_rep = pd.json_normalize(reps).reindex(columns=reindex_col)
    
    
    # (in REPLIES) retrive attached files and create table contains file name and download url
    files_rep = []
    for i in df_dm_rep.index:
        if 'files' in reps[i].keys():
            files_rep.append(reps[i]['files'][0])
        else:
            files_rep.append({'name': 'None', 'url_private_download': 'None'})  
    
    df_rep_files = pd.json_normalize(files)
    df_rep_files = df_rep_files[['name', 'url_private_download']]
    df_rep_files = df_rep_files.rename(columns={'name': 'FileName', 'url_private_download': 'FileURL'})
    print(df_rep_files)
    
    
    # combine REPLY table with FILE table and sort by thread_ts
    df_dm_rep_log = pd.concat([df_dm_rep, df_rep_files], axis=1).sort_values(by='thread_ts').reset_index().drop(columns='index')
    df_dm_rep_log['type']='thread'
    print(df_dm_rep_log)
    
    
    # combine MESSAGE with REPLY 
    df_dm_log = pd.concat([df_dm_log_ms, df_dm_rep_log])
    # remove duplicated parent messages by drop_duplicates
    df_dm_log = df_dm_log.drop_duplicates(subset = ['text', 'ts'], keep='last').reset_index().drop(columns='index')

else:
    df_dm_log = df_dm_log_ms

df_dm_log

## shape table

### convert dtypes of ts and thread_ts, from str to float, and replace NaN to 'None'

In [ ]:
df_dm_log_astype = df_dm_log.astype({'ts': float, 'thread_ts': float}).fillna('None')
df_dm_log_astype

### replace UNIX DATE to formatted one and separate ts to date and time (for multiindex)

In [ ]:
date = []
time = []
for i in range(len(df_dm_log_astype['ts'])):
    if (type(df_dm_log_astype['ts'][i]) == np.float64 or 
        type(df_dm_log_astype['ts'][i]) == float):
        # create list of date        
        dt_raw = datetime.date.fromtimestamp(df_dm_log_astype['ts'][i])
        dt = dt_raw.strftime('%a, %b %d, %Y')
        date.append(dt)
        print('finished formatting date')        
        
        # create list of time
        ti_raw = datetime.datetime.fromtimestamp(df_dm_log_astype['ts'][i])
        ti = ti_raw.strftime('%H:%M')
        time.append(ti)       
        print('finished formatting time') 

    # format thread_ts to datetime
    if (type(df_dm_log_astype['thread_ts'][i]) == np.float64 or 
        type(df_dm_log_astype['thread_ts'][i]) == float):
        dtime = datetime.datetime.fromtimestamp(df_dm_log_astype['thread_ts'][i])
        df_dm_log_astype.iloc[i, 1] = dtime.strftime('%H:%M, %a, %b %d, %Y')
        print ('finished formatting thread_datetime')
    
    else:
        print('thread_ts is not found')

        
# combine formatted date and time to dataframe
df_datetime = pd.DataFrame({'date': date, 
                            'time': time})
df_dm_log_frt = pd.concat([df_datetime, df_dm_log_astype],axis=1).drop(columns='ts')

### replace user id in 'names' column to 'real_name'

In [ ]:
for i in df_dm_log_frt.index:
    #print(i)
    for j in range(len(user_id_names)):
        if df_dm_log_frt.user[i] == df_user_id_names['id'][j]:
            df_dm_log_frt.user[i] = df_user_id_names['real_name'][j]

### replace user id in 'text' column to 'real_name' (maybe able to merge above cell?)

In [ ]:
for i in df_dm_log_frt.index:
    for j in df_user_id_names.index:
        if user_id_names[j][0] in df_dm_log_frt['text'][i]:
            df_dm_log_frt['text'][i] = df_dm_log_frt['text'][i].replace(df_user_id_names['id'][j], df_user_id_names['real_name'][j])
        else:
            continue

### for only uploading file (None of text fields)

In [ ]:
for i in df_dm_log_frt.index:
    if df_dm_log_frt.text[i] == '':
        df_dm_log_frt.text[i] = 'attached file(s) only'
    else:
        continue

### check result of formatting and replacing

In [ ]:
df_dm_log_frt

### multiIndex

In [ ]:
df_dm_log_frt_mi = df_dm_log_frt.set_index(['type', 'thread_ts', 'date', 'time'])
df_dm_log_frt_mi

## export files

### setting *you can custom something (year, path, etc.)

In [ ]:
my_name = df_user_id_names['real_name'][4] # or specify manually (ex. my_name = 'test')
print('your name: '+my_name+', dm_ch_user: '+dm_user_name)

In [ ]:
backup_date = datetime.date.today()
backup_date = backup_date.strftime('%y%m%d')

# DO NOT FORGET MODIFY YEAR (year of current backup workspace) 
tgt_ws_year = 2022
print(backup_date)
print(tgt_ws_year)

In [ ]:
path_DM = '../DM/'
if os.path.exists(path_DM) == False:
    %mkdir path_DM
else:
    print('directory already exists')
    
path_DM_full = '../DM_full_log/'
if os.path.exists(path_DM_full) == False:
    %mkdir path_DM_full
else:
    print('directory already exists')
    
path_YEAR = str(tgt_ws_year)
if os.path.exists(path_DM+path_YEAR) == False:
    %mkdir path_DM+path_YEAR
else:
    print('directory already exists')

if os.path.exists(path_DM_full+path_YEAR) == False:
    %mkdir path_DM_full+path_YEAR
else:
    print('directory already exists')

### export excel file (for remaining multiIndex and encoding automatically)

In [ ]:
df_dm_log_frt_mi.to_excel(path_DM+path_YEAR+'/DM_log_'+path_YEAR+'_'+my_name+'_'+dm_user_name+'_'+backup_date+'.xlsx')

### export raw file (json format)

#### not included replies

In [ ]:
# recommend saving raw data (not included replies)
with open(path_DM_full+path_YEAR+'/Raw_dm_log_NOrep'+path_YEAR+'_'+my_name+'_'+dm_user_name+'_'+backup_date+'_.dat', 'w') as f:
    print(log_dm, file=f) 

#### replies data

In [ ]:
with open(path_DM_full+path_YEAR+'/Raw_dm_rep_log'+path_YEAR+'_'+my_name+'_'+dm_user_name+'_'+backup_date+'_.dat', 'w') as f:
    print(thr, file=f) 